# Гришин Андрей Валерьевич, ИКБО-11-22.
## Практическая работа 7

# Задание 1
Найти данные для задачи классификации или для задачи регрессии
(данные не должны повторятся в группе)

In [60]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

data = pd.read_csv('CKD_NephrotoxicDrug_Dataset.csv') # https://www.kaggle.com/datasets/ziya07/drugpatient-dataset-for-ckd-prediction

X = data.drop(columns=["ckd_risk_label"])
y = data["ckd_risk_label"]

# Обработка категориальных признаков
cat_cols = ["gender", "drug_name"]
encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    encoders[col] = le

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(
    f"Данные загружены. "
    f"Обучающая выборка: {X_train.shape}, "
    f"Тестовая: {X_test.shape}\n"
)

Данные загружены. Обучающая выборка: (1200, 36), Тестовая: (300, 36)



# Задание 2
Реализовать баггинг

In [61]:
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

start_rf = time.perf_counter()

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)

time_rf = time.perf_counter() - start_rf
acc_rf = accuracy_score(y_test, rf_pred)

print(f"Баггинг завершен за {time_rf:.4f} сек. Точность: {(acc_rf * 100):.5f}%")

Баггинг завершен за 0.4597 сек. Точность: 86.33333%


# Задание 3
Реализовать бустинг на тех же данных, что использовались для баггинга

In [62]:
import catboost as cb

start_cb = time.perf_counter()

cb_model = cb.CatBoostClassifier(iterations=1000, learning_rate=0.05, random_seed=42, verbose=0)
cb_model.fit(X_train, y_train)
cb_pred = cb_model.predict(X_test)

time_cb = time.perf_counter() - start_cb
acc_cb = accuracy_score(y_test, cb_pred)

print(f"Бустинг завершен за {time_cb:.4f} сек. Точность: {(acc_cb * 100):.5f}%")

Бустинг завершен за 7.1689 сек. Точность: 100.00000%


# Задание 4
Сравнить результаты работы алгоритмов (время работы и качество
моделей). Сделать выводы

In [63]:
from sklearn.metrics import f1_score
from tabulate import tabulate

table = [
    ["Random Forest",
     accuracy_score(y_test, rf_pred),
     f1_score(y_test, rf_pred, average="weighted"),
     time_rf],

    ["Cat Boosting",
     accuracy_score(y_test, cb_pred),
     f1_score(y_test, cb_pred, average="weighted"),
     time_cb]
]
headers = ["Алгоритм", "Accuracy", "F1-score", "Время (сек)"]

print(tabulate(table, headers=headers, floatfmt=".5f"))

Алгоритм         Accuracy    F1-score    Время (сек)
-------------  ----------  ----------  -------------
Random Forest     0.86333     0.86317        0.45966
Cat Boosting      1.00000     1.00000        7.16885
